In [1]:
import os
import random
import copy
import matplotlib.pyplot as plt
import statsmodels as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
import pandas as pd
import fix_yahoo_finance as yf
import numpy as np
from numpy import std
from statistics import stdev
from sklearn.preprocessing import MinMaxScaler
import talib as tb
from sklearn import svm

In [27]:
# SVM:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import recall_score, f1_score, precision_score

In [2]:
# LSTM:
from keras.models import Sequential
from keras.layers import Dense, LSTM

Using TensorFlow backend.


In [3]:
# Significado das entradas da função 'leitura': leitura padrão
# diretorio: digitar entre aspas o diretório em que os dados históricos estão hospedados. Exemplo:'/home/giovana/Área de Trabalho/Implementações/Projeto/Dados/sITE/MGLU3.SA.csv'
# Chamando a função:
# leitura('/home/giovana/Área de Trabalho/Implementações/Projeto/Dados/sITE/MGLU3.SA.csv')

def leitura(diretorio):
    serie = pd.read_csv(diretorio)
    return serie

# Para visualizar um número especícico de linhas, defina uma variável como a função 'leitura' e aplique o método '.head()'
# Exemplo:
# >>> x = leitura('/home/giovana/Área de Trabalho/Implementações/Projeto/Dados/sITE/MGLU3.SA.csv')
# >>> x.head(15)

In [4]:
# Significado das entradas da função 'removendo_valores_discrepantes': remover as linhas que apresentem como valores '0', 'Nan' ou 'null'
# dados_historicos: recomenda-se definir a função leitura como uma variável que pode ser interpretada como parâmetro. Um exemplo é a variável 'dados'.
# Chamando a função:
# removendo_volume_zerado(dados)

def removendo_valores_discrepantes(dados_historicos):
    dados_historicos = dados_historicos[dados_historicos['Volume']>0]
    return dados_historicos

In [5]:
def removendo_dados_duplicados(dados_historicos):
    dados_historicos = dados_historicos.drop_duplicates()
    return dados_historicos

In [6]:
def remover_coluna_date(dados_historicos):
    dados_sem_data = dados_historicos.drop(columns=['Date'])
    return dados_sem_data

# Gerando os indicadores técnicos

- __Médias móveis:__ suaviza os preços anteriores para identificar a movimentação dos preços.
    - Média móvel(simples e exponencial).
- __Tendência:__ indica pontos de reversão de tendência.
    - MACD.
- __Momento:__ acompanha a taxa de variação dos preços.
    - IFR.
- __Volatilidade:__ identifica se as variações de preço futuras estão seguindo o mesmo ritmo das variações passadas.
    - Bandas de Bollinger.
- __Volume:__ determina se a tendência é relevante.
    - Osciladores de Chaikin

In [7]:
def MMS(dados_historicos, periodo):
    #Criando uma janela dos elementos considerados:
    janela = dados_historicos['Close'].rolling(window= periodo, center = True)#Decidir se será centrado
    #Calculando a média da janela:
    media_movel = janela.mean()
    dados_historicos['MMS'] = media_movel
    
    #Problema do Nan:
    dados_historicos['MMS'].values[:int(periodo/2)+1]= dados_historicos['MMS'].values[int(periodo/2)+1]
    dados_historicos['MMS'].values[-(int(periodo/2)-1):]= dados_historicos['MMS'].values[-(int(periodo/2))]
    
    #Classificando a média móvel simples:
    dif = dados_historicos['MMS'].values[1:]-dados_historicos['MMS'].values[:-1]
    dif = np.concatenate([[np.nan], dif])
    dados_historicos['DIF_MMS']= dif
    dados_historicos['ClasseMMS']= (dados_historicos['DIF_MMS']>0).astype(int)
    dados_historicos['ClasseMMS'].values[:1] = 0
    return dados_historicos

In [8]:
def MME(dados_historicos, periodo, inicio_do_cálculo):
    mme = dados_historicos['Close'].ewm(span = periodo, min_periods = inicio_do_cálculo, adjust = True).mean()
    dados_historicos['MME'] = mme
    
    #Problema do Nan:
    dados_historicos['MME'].values[:inicio_do_cálculo-1]= dados_historicos['MME'].values[inicio_do_cálculo-1]
    
    #Classificando a média móvel exponencial:
    dif_1 = dados_historicos['MME'].values[1:]-dados_historicos['MME'].values[:-1]
    dif_1 = np.concatenate([[np.nan], dif_1])
    dados_historicos['DIF_MME']= dif_1
    dados_historicos['ClasseMME']= (dados_historicos['DIF_MME']>0).astype(int)
    dados_historicos['ClasseMME'].values[:1] = 0
    return dados_historicos

In [9]:
def MACD (dados_historicos):
    #parâmetros da linha macd
    mme26 = dados_historicos['Close'].ewm(span = 26, min_periods = 0, adjust = True).mean()
    dados_historicos['MME26']=mme26
    mme12 = dados_historicos['Close'].ewm(span = 12, min_periods = 0, adjust = True).mean()
    dados_historicos['MME12']=mme12
    #linha macd
    macd = dados_historicos['MME12'] - dados_historicos['MME26']
    dados_historicos['MACD'] = macd
    
    #linha de sinal: mme(9)
    #mme9 = dados_historicos['Close'].ewm(span = 9, min_periods = 0, adjust = True).mean()
    #dados_historicos['MME9']=mme9
    
    #Classificando o macd:
    dados_historicos['ClasseMACD'] = (dados_historicos['MACD']>0).astype(int)
    return dados_historicos

In [10]:
# O padrão é utilizar 14 dias, mas também se usa 9 ou 25. 

def RSI(dados_historicos, periodo):
    dados_historicos['RSI'] = tb.RSI(dados_historicos['Close'] , periodo)
    
    #Classificação:
    dados_historicos['ClasseRSI']=1
    dados_historicos.loc[dados_historicos['RSI']<50,'ClasseRSI']=0
    
    #Problema do Nan:
    dados_historicos['RSI'].values[:periodo]= dados_historicos['RSI'].values[periodo]
    
    return dados_historicos

In [11]:
def target(dados_historicos):
    soma = dados_historicos['ClasseMMS']+dados_historicos['ClasseMME']+dados_historicos['ClasseMACD']+dados_historicos['ClasseRSI']
    dados_historicos['soma'] = soma
    dados_historicos['Soma'] = (dados_historicos['soma']>2).astype(int)
    return dados_historicos

In [12]:
def target_treinamento(dados_historicos):
    soma = dados_historicos['ClasseMMS']+dados_historicos['ClasseMME']+dados_historicos['ClasseMACD']+dados_historicos['ClasseRSI']
    dados_historicos['soma'] = soma
    dados_historicos['Soma'] = (dados_historicos['soma']>2).astype(int)
    classe = pd.DataFrame()
    classe['target'] = dados_historicos['Soma'].values[:int(len(dados_historicos)*0.7)+1]
    return classe

In [13]:
def  conjunto_indicadores_treinamento(dados_historicos):
    classe = pd.DataFrame()
    classe['MMS'] = dados_historicos['MMS'].values[:int(len(dados_historicos)*0.7)+1]
    classe['MME'] = dados_historicos['MME'].values[:int(len(dados_historicos)*0.7)+1]
    classe['MACD'] = dados_historicos['MACD'].values[:int(len(dados_historicos)*0.7)+1]
    classe['RSI'] = dados_historicos['RSI'].values[:int(len(dados_historicos)*0.7)+1]
    return classe

In [14]:
def  conjunto_indicadores_teste(dados_historicos):
    classe_teste = pd.DataFrame()
    classe_teste['MMS'] = dados_historicos['MMS'].values[1+int(len(dados_historicos)*0.7):]
    classe_teste['MME'] = dados_historicos['MME'].values[1+int(len(dados_historicos)*0.7):]
    classe_teste['MACD'] = dados_historicos['MACD'].values[1+int(len(dados_historicos)*0.7):]
    classe_teste['RSI'] = dados_historicos['RSI'].values[1+int(len(dados_historicos)*0.7):]
    return classe_teste

In [15]:
def target_teste(dados_historicos):
    soma = dados_historicos['ClasseMMS']+dados_historicos['ClasseMME']+dados_historicos['ClasseMACD']+dados_historicos['ClasseRSI']
    dados_historicos['soma'] = soma
    dados_historicos['Soma'] = (dados_historicos['soma']>2).astype(int)
    classe = pd.DataFrame()
    classe['target'] = dados_historicos['Soma'].values[int(len(dados_historicos)*0.7)+1:]
    return classe

In [16]:
def normalizador(indicadores):
    indicadores = MinMaxScaler(feature_range=(0,1)).fit_transform(indicadores)
    return indicadores

In [17]:
def modelo(C, kernel, gamma, vetor_caracteristica_treinamento, rotulo_treinamento, vetor_caracteristica_teste):
    model = svm.SVC(C = C, kernel= kernel, gamma= gamma) 
    model.fit(vetor_caracteristica_treinamento, rotulo_treinamento)
    predicted= model.predict(vetor_caracteristica_teste)
    return predicted

In [18]:
def reshape_target(target):
    target_reshape = target.reshape(len(target), )
    return target_reshape

A acurácia (A) corresponde à taxa de acertos do modelo e relaciona o total de entradas previdas corretamente com o total de previsões

$$
A = \frac{t_p + t_n}{t_p + t_n + f_p + f_n}
$$


A precisão (P) corresponde à taxa de acerto de previsões positivas e relaciona os verdadeiros positivos com o total de positivos previstos

$$P = \frac{t_p}{t_p + f_p}$$


A métrica recall (R) indica o número de valores previstos corretamente dentro de uma classe relacionando o número de previsões corretas em uma classe e o número total de ocorrências dela e, portanto, pode ser calculado para ambas as classes
$$R =\frac{t_p}{t_p + f_n}$$

A F1-score representa a média harmônica entre as medidas de precisão e recall
$$F1 = \frac{2RP}{P + R}$$

In [19]:
def relatorio(previsao_do_modelo, target_teste):
    matriz_confusão = confusion_matrix(previsao_do_modelo,target_teste)
    
    
    rel = pd.DataFrame()
    rel['Falsos positivo - fp'] = [matriz_confusão[0,1]]
    rel['Verdadeiros positivo - tp'] = [matriz_confusão[1,1]]
    rel['Falsos negativo - fn'] = [matriz_confusão[1,0]]
    rel['Verdadeiros negativo  - tn'] = [matriz_confusão[0,0]]
    rel['Recall'] = [recall_score(target_teste, previsao_do_modelo)]
    rel['F1-score'] = [f1_score(target_teste, previsao_do_modelo)]
    rel['Precisão'] = [precision_score(target_teste, previsao_do_modelo)]
    rel['Acurácia'] = [accuracy_score(target_teste, previsao_do_modelo)]
    return rel
    

### GridSearchCV 

 O GridSearchCV pega um dicionário que descreve os parâmetros que podem ser experimentados em um modelo para treiná-lo. A grade de parâmetros é definida como um dicionário, onde as chaves são os parâmetros e os valores são as configurações a serem testadas.

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
def model_grid(x_treinamento, x_teste, y_treinamento, y_teste):
    # Definindo os parâmetros:
    parametros_grid = {'C':  [0.1,1, 10, 100], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf', 'poly', 'linear', 'sigmoid']} 
    # Criando o modelo em que a grade de parâmetros será testada:
    grid = GridSearchCV(SVC(), parametros_grid, refit = True, verbose = 3)
    # Treinando o modelo:
    grid.fit(x_treinamento, y_treinamento)
    # Imprimindo os melhores parâmetros:
    print('Melhores parâmetros')
    print(grid.best_params_)
    # Imprimindo as configurações do melhor estimador:
    print('Melhor estimador')
    print(grid.best_estimator_)
    # Realizando as predições:
    grid_predictions = grid.predict(x_teste)
    # Imprimindo as métricas de avaliação:
    print(classification_report(y_teste, grid_predictions))
    
    
    
    matriz_confusão = confusion_matrix(grid_predictions,y_teste)
    print(f'Falsos positivo - fp: {matriz_confusão[0,1]}')
    print(f'Verdadeiros positivo - tp: {matriz_confusão[1,1]}')
    print(f'Falsos negativo - fn: {matriz_confusão[1,0]}')
    print(f'Verdadeiros negativo  - tn: {matriz_confusão[0,0]}')
    print(f'Recall: {recall_score(y_teste, grid_predictions)}')
    print(f'F1-score: {f1_score(y_teste, grid_predictions)}')
    print(f'Precisão: {precision_score(y_teste, grid_predictions)}')
    print(f'Acurácia: {accuracy_score(y_teste, grid_predictions)}')

    

# LSTM

In [1]:
def lstm1(indicadores_treinamento, indicadores_teste, target_treinamento, target_teste):
    
    # Arrumando a entrada do conjunto de treinamento no formato exigido pelo LSTM
    xtreino = indicadores_treinamento
    xtreino_novo = xtreino.reshape((xtreino.shape[0],xtreino.shape[1],1))
    
    # Arrumando a entrada do conjunto de teste no formato exigido pelo LSTM
    xteste = indicadores_teste
    xteste_novo = xteste.reshape((xteste.shape[0],xteste.shape[1],1))
    
    # Modelo 1:
    model = Sequential()
    model.add(LSTM(50, activation= 'relu', input_shape = (xtreino_novo.shape[1],xtreino_novo.shape[2])))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Resumo da Rede Neural:
    print('Resumo da Rede Neural:')
    model.summary()
    print('---------------------------------------------')
    
    # Ajuste da Rede Neural ao conjunto de treinamento:
    model.fit(xtreino_novo, target_treinamento, epochs = 10, batch_size = 1)
    
    # Ajustando o modelo ao conjunto de entrada de treinamento:
    yajustadolstm = model.predict_classes(xtreino_novo)
    # Avaliando a previsão:
    print('---------------------------------------------')
    print('Ajustando o modelo ao conjunto de entrada de treinamento')
    matriz_confusão = confusion_matrix(yajustadolstm,target_treinamento)
    print(f'Falsos positivo - fp: {matriz_confusão[0,1]}')
    print(f'Verdadeiros positivo - tp: {matriz_confusão[1,1]}')
    print(f'Falsos negativo - fn: {matriz_confusão[1,0]}')
    print(f'Verdadeiros negativo  - tn: {matriz_confusão[0,0]}')
    print(f'Recall: {recall_score(target_treinamento, yajustadolstm)}')
    print(f'F1-score: {f1_score(target_treinamento, yajustadolstm)}')
    print(f'Precisão: {precision_score(target_treinamento, yajustadolstm)}')
    print(f'Acurácia: {accuracy_score(target_treinamento, yajustadolstm)}')
    
    # Ajustando o modelo ao conjunto de entrada de teste: 
    y_predict = model.predict_classes(xteste_novo)
    # Avaliando a previsão:
    print('---------------------------------------------')
    print('Ajustando o modelo ao conjunto de entrada de teste')
    matriz_confusão = confusion_matrix(y_predict,target_teste)
    print(f'Falsos positivo - fp: {matriz_confusão[0,1]}')
    print(f'Verdadeiros positivo - tp: {matriz_confusão[1,1]}')
    print(f'Falsos negativo - fn: {matriz_confusão[1,0]}')
    print(f'Verdadeiros negativo  - tn: {matriz_confusão[0,0]}')
    print(f'Recall: {recall_score(target_teste, y_predict)}')
    print(f'F1-score: {f1_score(target_teste, y_predict)}')
    print(f'Precisão: {precision_score(target_teste, y_predict)}')
    print(f'Acurácia: {accuracy_score(target_teste, y_predict)}')

In [6]:
def lstm2(indicadores_treinamento, indicadores_teste, target_treinamento, target_teste):
    
    # Arrumando a entrada do conjunto de treinamento no formato exigido pelo LSTM
    xtreino = indicadores_treinamento
    xtreino_novo = xtreino.reshape((xtreino.shape[0],xtreino.shape[1],1))
    
    # Arrumando a entrada do conjunto de teste no formato exigido pelo LSTM
    xteste = indicadores_teste
    xteste_novo = xteste.reshape((xteste.shape[0],xteste.shape[1],1))
    
    # Modelo 2:
    model = Sequential()
    model.add(LSTM(10, activation= 'relu', return_sequences = True, input_shape = (xtreino_novo.shape[1],xtreino_novo.shape[2])))
    model.add(LSTM(10, activation= 'relu', return_sequences = True))
    model.add(LSTM(10, activation= 'relu', return_sequences = True))
    model.add(LSTM(10, activation= 'relu', return_sequences = True))
    model.add(LSTM(10, activation= 'relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Resumo da Rede Neural:
    print('Resumo da Rede Neural:')
    model.summary()
    print('---------------------------------------------')
    
    # Ajuste da Rede Neural ao conjunto de treinamento:
    model.fit(xtreino_novo, target_treinamento, epochs = 10, batch_size = 1)
    
    # Ajustando o modelo ao conjunto de entrada de treinamento:
    yajustadolstm = model.predict_classes(xtreino_novo)
    # Avaliando a previsão:
    print('---------------------------------------------')
    print('Ajustando o modelo ao conjunto de entrada de treinamento')
    matriz_confusão = confusion_matrix(yajustadolstm,target_treinamento)
    print(f'Falsos positivo - fp: {matriz_confusão[0,1]}')
    print(f'Verdadeiros positivo - tp: {matriz_confusão[1,1]}')
    print(f'Falsos negativo - fn: {matriz_confusão[1,0]}')
    print(f'Verdadeiros negativo  - tn: {matriz_confusão[0,0]}')
    print(f'Recall: {recall_score(target_treinamento, yajustadolstm)}')
    print(f'F1-score: {f1_score(target_treinamento, yajustadolstm)}')
    print(f'Precisão: {precision_score(target_treinamento, yajustadolstm)}')
    print(f'Acurácia: {accuracy_score(target_treinamento, yajustadolstm)}')
    
    # Ajustando o modelo ao conjunto de entrada de teste: 
    y_predict = model.predict_classes(xteste_novo)
    # Avaliando a previsão:
    print('---------------------------------------------')
    print('Ajustando o modelo ao conjunto de entrada de teste')
    matriz_confusão = confusion_matrix(y_predict,target_teste)
    print(f'Falsos positivo - fp: {matriz_confusão[0,1]}')
    print(f'Verdadeiros positivo - tp: {matriz_confusão[1,1]}')
    print(f'Falsos negativo - fn: {matriz_confusão[1,0]}')
    print(f'Verdadeiros negativo  - tn: {matriz_confusão[0,0]}')
    print(f'Recall: {recall_score(target_teste, y_predict)}')
    print(f'F1-score: {f1_score(target_teste, y_predict)}')
    print(f'Precisão: {precision_score(target_teste, y_predict)}')
    print(f'Acurácia: {accuracy_score(target_teste, y_predict)}')

In [7]:
def lstm3(indicadores_treinamento, indicadores_teste, target_treinamento, target_teste):
    
    # Arrumando a entrada do conjunto de treinamento no formato exigido pelo LSTM
    xtreino = indicadores_treinamento
    xtreino_novo = xtreino.reshape((xtreino.shape[0],xtreino.shape[1],1))
    
    # Arrumando a entrada do conjunto de teste no formato exigido pelo LSTM
    xteste = indicadores_teste
    xteste_novo = xteste.reshape((xteste.shape[0],xteste.shape[1],1))
    
    # Modelo 3:
    model = Sequential()
    model.add(LSTM(12, activation='relu', return_sequences = True, input_shape = (xtreino_novo.shape[1],xtreino_novo.shape[2])))
    model.add(LSTM(8, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Resumo da Rede Neural:
    print('Resumo da Rede Neural:')
    model.summary()
    print('---------------------------------------------')
    
    # Ajuste da Rede Neural ao conjunto de treinamento:
    model.fit(xtreino_novo, target_treinamento, epochs = 10, batch_size = 1)
    
    # Ajustando o modelo ao conjunto de entrada de treinamento:
    yajustadolstm = model.predict_classes(xtreino_novo)
    # Avaliando a previsão:
    print('---------------------------------------------')
    print('Ajustando o modelo ao conjunto de entrada de treinamento')
    matriz_confusão = confusion_matrix(yajustadolstm,target_treinamento)
    print(f'Falsos positivo - fp: {matriz_confusão[0,1]}')
    print(f'Verdadeiros positivo - tp: {matriz_confusão[1,1]}')
    print(f'Falsos negativo - fn: {matriz_confusão[1,0]}')
    print(f'Verdadeiros negativo  - tn: {matriz_confusão[0,0]}')
    print(f'Recall: {recall_score(target_treinamento, yajustadolstm)}')
    print(f'F1-score: {f1_score(target_treinamento, yajustadolstm)}')
    print(f'Precisão: {precision_score(target_treinamento, yajustadolstm)}')
    print(f'Acurácia: {accuracy_score(target_treinamento, yajustadolstm)}')
    
    # Ajustando o modelo ao conjunto de entrada de teste: 
    y_predict = model.predict_classes(xteste_novo)
    # Avaliando a previsão:
    print('---------------------------------------------')
    print('Ajustando o modelo ao conjunto de entrada de teste')
    matriz_confusão = confusion_matrix(y_predict,target_teste)
    print(f'Falsos positivo - fp: {matriz_confusão[0,1]}')
    print(f'Verdadeiros positivo - tp: {matriz_confusão[1,1]}')
    print(f'Falsos negativo - fn: {matriz_confusão[1,0]}')
    print(f'Verdadeiros negativo  - tn: {matriz_confusão[0,0]}')
    print(f'Recall: {recall_score(target_teste, y_predict)}')
    print(f'F1-score: {f1_score(target_teste, y_predict)}')
    print(f'Precisão: {precision_score(target_teste, y_predict)}')
    print(f'Acurácia: {accuracy_score(target_teste, y_predict)}')

In [9]:
def lstm4(indicadores_treinamento, indicadores_teste, target_treinamento, target_teste):
    
    # Arrumando a entrada do conjunto de treinamento no formato exigido pelo LSTM
    xtreino = indicadores_treinamento
    xtreino_novo = xtreino.reshape((xtreino.shape[0],xtreino.shape[1],1))
    
    # Arrumando a entrada do conjunto de teste no formato exigido pelo LSTM
    xteste = indicadores_teste
    xteste_novo = xteste.reshape((xteste.shape[0],xteste.shape[1],1))
    
    # Modelo 4:
    model = Sequential()
    model.add(LSTM(30, activation= 'relu', input_shape = (xtreino_novo.shape[1],xtreino_novo.shape[2])))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Resumo da Rede Neural:
    print('Resumo da Rede Neural:')
    model.summary()
    print('---------------------------------------------')
    
    # Ajuste da Rede Neural ao conjunto de treinamento:
    model.fit(xtreino_novo, target_treinamento, epochs = 10, batch_size = 1)
    
    # Ajustando o modelo ao conjunto de entrada de treinamento:
    yajustadolstm = model.predict_classes(xtreino_novo)
    # Avaliando a previsão:
    print('---------------------------------------------')
    print('Ajustando o modelo ao conjunto de entrada de treinamento')
    matriz_confusão = confusion_matrix(yajustadolstm,target_treinamento)
    print(f'Falsos positivo - fp: {matriz_confusão[0,1]}')
    print(f'Verdadeiros positivo - tp: {matriz_confusão[1,1]}')
    print(f'Falsos negativo - fn: {matriz_confusão[1,0]}')
    print(f'Verdadeiros negativo  - tn: {matriz_confusão[0,0]}')
    print(f'Recall: {recall_score(target_treinamento, yajustadolstm)}')
    print(f'F1-score: {f1_score(target_treinamento, yajustadolstm)}')
    print(f'Precisão: {precision_score(target_treinamento, yajustadolstm)}')
    print(f'Acurácia: {accuracy_score(target_treinamento, yajustadolstm)}')
    
    # Ajustando o modelo ao conjunto de entrada de teste: 
    y_predict = model.predict_classes(xteste_novo)
    # Avaliando a previsão:
    print('---------------------------------------------')
    print('Ajustando o modelo ao conjunto de entrada de teste')
    matriz_confusão = confusion_matrix(y_predict,target_teste)
    print(f'Falsos positivo - fp: {matriz_confusão[0,1]}')
    print(f'Verdadeiros positivo - tp: {matriz_confusão[1,1]}')
    print(f'Falsos negativo - fn: {matriz_confusão[1,0]}')
    print(f'Verdadeiros negativo  - tn: {matriz_confusão[0,0]}')
    print(f'Recall: {recall_score(target_teste, y_predict)}')
    print(f'F1-score: {f1_score(target_teste, y_predict)}')
    print(f'Precisão: {precision_score(target_teste, y_predict)}')
    print(f'Acurácia: {accuracy_score(target_teste, y_predict)}')